# Blockchain

Pour commencer, on va définir une structure de block simple en python.

Le bloc prendra pour argument une timestamp, des données et le hash du dernier block.

In [2]:
import hashlib
from datetime import datetime

class Block:
    def __init__(self, timestamp, data, last_block=None):
        self.timestamp = timestamp
        self.data = data
        self.last_block = last_block

    @property
    def hash(self):
        m = hashlib.sha256()
        m.update(bytes(str(self.timestamp), "utf-8"))
        m.update(bytes(self.data, "utf-8"))
        if self.last_block:
            m.update(bytes(self.last_block.hash, "utf-8"))
        return m.hexdigest()

Dans cet exemple de blog nous utilisons la fonction `sha256` de la librairie `hashlib` de Python. Cette fonction nous permettra de créer le hash du bloc.

Les fonctions de hashage (hashing functions) sont des algorithmes qui acceptent n'importe quel donnée de nimporte quelle taille et génère une chaine de caractère de longueur fixe en sortie.

In [4]:
print(hashlib.sha256(b"Salut!").hexdigest())
print(hashlib.sha256(b"Salut tout le monde, vous allez bien?").hexdigest())

e3c18f5c1f8a09f3353e25bde7bfbbb731209ce72dffd2720f85ec11a06b9155
602d108f618eb96268bcef7b97b2b318a73fd4221b23f0c4db615db518943ce2


Les fonctions de hashage sont déterministes. Cela signifie que si l'on les lances plusieurs fois de suite, le résultat sera identique

In [5]:
print(hashlib.sha256(b"Salut!").hexdigest())
print(hashlib.sha256(b"Salut!").hexdigest())

e3c18f5c1f8a09f3353e25bde7bfbbb731209ce72dffd2720f85ec11a06b9155
e3c18f5c1f8a09f3353e25bde7bfbbb731209ce72dffd2720f85ec11a06b9155


Un très petit changement dans la chaîne d'entrée crée un hash totalement différent.

In [6]:
print(hashlib.sha256(b"Salut!").hexdigest())
print(hashlib.sha256(b"salut!").hexdigest())


e3c18f5c1f8a09f3353e25bde7bfbbb731209ce72dffd2720f85ec11a06b9155
1bfdc23d3e2901d307eadaeb8fb299f2a914a2d80d785a6088ef8bcf6d067e47


On ne peut pas remonter aux données d'origine à partir du hash.

Ces propriétés font du hash un outil très puissant pour vérifier l'intégrité de données.

On va définir un bloc genesis, le premier de la blockchain.

In [28]:
genesis = Block(
    datetime.timestamp(datetime.now()),
    "Salut tout le monde, je suis le premier bloc !"
)

print(f"genesis block hash: {genesis.hash}")

genesis block hash: f20ee5c9440f16c19e5800aab8759b2af67d022deba007af5aa26b55c4f8f454


Les blocs suivants prendront le hash du block précédent comme argument

In [35]:
block1 = Block(
    datetime.timestamp(datetime.now()),
    "On ajoute des données dans la blockchain",
    last_block=genesis
)
block2 = Block(
    datetime.timestamp(datetime.now()),
    "encore",
    last_block=block1
)

block3 = Block(
    datetime.timestamp(datetime.now()),
    "et encore",
    last_block=block2
)

print(f"block1 hash: {block1.hash}")
print(f"block2 hash: {block2.hash}")
print(f"block3 hash: {block3.hash}")

block1 hash: e43ca6fb90bb0e4cd9cb86b2a2cf0d97e6d0d8db8fa60b1ed5607973d645a08e
block2 hash: 9f78c02e197d5de658d3bcc6df87e24186243df93e1106a129949deb2346893a
block3 hash: f6396d8d081748fee22a70994173f72cad52189fd129c455da3e24c734d56107


Maintenant imaginons que je change une donnée dans un des blocs précédents:

In [36]:
print(f"block3 hash: {block3.hash}")

block1.data = "je modifie les données, ni vu ni connu..."

print(f"block3 hash: {block3.hash}")

block3 hash: f6396d8d081748fee22a70994173f72cad52189fd129c455da3e24c734d56107
block3 hash: ddbbb486b6f4d4ff63a562424abd4b8eddb54ea4a395c9710337f22962cdcec6


Le hash du block a changé ! Dans un système à blockchain, si quelqu'un modifie le contenu d'un block antérieur, toute la chaine de hash se retrouve modifiée jusqu'au dernier block.

En vérifiant le hash du block précédent, on peut facilement déterminer si les données ont été changées sans avoir à vérifier toute les transactions une à une. Si un hash n'a pas la valeur attendue, alors les données ont probablement été corrompus ou altérés.
